In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
from itertools import permutations
import networkx as nx

In [4]:
A = np.array([[1, 1, 1],[-1,0,0],[0,1,0]])
nx.from_numpy_matrix(A)

In [64]:
#Exhaustively list all possible configurations of the 3x3 adjacency matrix
def find_ternary(num): 
    quotient = num/3    
    remainder = num%3
    if quotient == 0:   
        return ""
    else:
        return find_ternary(int(quotient)) + str(int(remainder))

adj_mats = np.zeros((19683,3,3)) #master list
for i in range(19683):
    ternary = np.array(list(map(int, find_ternary(i))))
    adj_mats[i]=np.reshape(np.pad(ternary,(9-len(ternary),0),'constant'),(3,3))
    adj_mats[i] = adj_mats[i] - np.ones((3,3))
    

In [306]:
#How many of the motifs fit the requirement of the output neuron being self-inhibitory and having exactly one incoming connection which is excitatory?
foo = 0
for i in range(19683):
    if adj_mats[i,2,2] == -1: #inhibitory self-connection
        if adj_mats[i,0,2] + adj_mats[i,1,2] == 1: #exactly one incoming excitatory connection
            foo += 1
print(foo)

1458


In [139]:
def lookupclass(matrices,matrix):
    for i in range(len(matrices)):
        for j in range(len(matrices[i])):
            if (matrices[i,j]==matrix).all():
                return i
    else: #if you can't find that matrix in the list of matrices
        return "n"

In [169]:
#partition into equivalence classes
partition = np.ones((3411,100,3,3))*3 #make it full of threes so that you can't falsely locate the zeros matrix everywhere

rearrangements = [[0,1,2],[0,2,1],[1,0,2],[1,2,0],[2,1,0],[2,0,1]]#list of possible equivalant neuron rearrangements (including original)

classcounter = 0 #keep track of what class we're on 
for i in range(len(adj_mats)):
    #print(adj_mats[i])
    if lookupclass(partition,adj_mats[i]) != 'n': #if we can already locate the matrix in the partition
        pass
    else: #if we can't
        elementcounter = 0 #keep track of where in the class we're adding
        for j in range(6):
            test = adj_mats[i][rearrangements[j],:] #rearrange rows
            test = test[:,rearrangements[j]] #rearrange columns
            if lookupclass(partition, test) == 'n':
                partition[classcounter,elementcounter]=test #and each of its equivalent permutations one by one
                elementcounter+=1
        classcounter += 1
        


In [279]:
#Lookup of a threegroup

genome=np.reshape(and_genomes[10],(5,5))
new = genome[2:5,2:5]
new1 = np.ones((3,3))
print(new)
lookupclass(partition,new)


[[ 1.  1.  0.]
 [ 1.  1. -1.]
 [-1. -1. -1.]]


188

In [281]:
visualizeclass(1440,rep_classmembers) 

test1 = np.array([[0,1,0],[0,0,1],)

[0. 1. 0.],[0. 0. 1.],[-1. -1. -1.]
[0. 1. 0.],[0. 0. 1.],[ 0. -1. -1.]
[0. 1. 0.],[ 1. -1.  1.],[-1.  0. -1.]
[0. 1. 0.],[1. 0. 1.],[-1. -1. -1.]
[0. 1. 1.],[ 1. -1.  0.],[-1. -1. -1.]

[[-1. -1.  1.]
 [ 1.  0.  0.]
 [ 0.  1. -1.]]


In [186]:
#scratch cell
#a = np.tile([1,2,3,4,5],(4,1))
a = np.array([[0,-1,0,1],[0,0,-1,1],[-1,1,1,1],[1,1,1,1]])
new = a[[1,2,3]]
new = new[:,[1,2,3]]

In [232]:
#for each successful genome, test every combination of three cells and collect its motif class
#output the counts of each as a 3411 element array

#every combination of three indices, from 0 to 24 (for a 5-neuron CTRNN)
combs = list(permutations(np.arange(5),3))

def motifcount(genome): 
    weightmatrix = np.reshape(genome,(5,5))
    motifdist = np.zeros(3411)
    for i in range(len(combs)): 
        comb = list(combs[i])
        threegroup = weightmatrix[comb] #remove unnecessary rows
        threegroup = threegroup[:,comb]  #remove unnecessary columns
        motif = lookupclass(partition,threegroup)
        motifdist[motif] += 1
    return motifdist     


In [250]:
#load in genome data sets and run counter
random_genomes = np.genfromtxt('59randomgenomes.csv', delimiter=',')
countdata = np.zeros((len(random_genomes),3411))
for i in range(len(random_genomes)):
    countdata[i] = motifcount(random_genomes[i])
    print(random_genomes[i],'done')

[ 0. -1. -1. -1. -1.  1.  1.  0.  1. -1.  0.  1. -1.  0. -1.  0. -1. -1.
  0.  0.  1. -1. -1. -1. -1.] done
[ 1.  1.  0.  1. -1. -1. -1. -1.  1.  1.  0. -1. -1.  1. -1.  1. -1.  1.
  1. -1.  0. -1.  0.  1.  0.] done
[-1.  0. -1.  0.  1. -1.  1. -1. -1.  1. -1.  1.  0. -1. -1.  0. -1. -1.
  1.  0. -1.  1.  0.  0. -1.] done
[-1.  1.  1. -1.  0.  1. -1.  0.  1. -1.  0.  0. -1.  0.  1.  0.  1. -1.
 -1.  0.  1.  0.  1.  0. -1.] done
[-1. -1. -1. -1. -1. -1.  1.  0. -1.  1. -1.  1.  1. -1. -1.  1.  0.  0.
  1.  0. -1.  1.  0. -1.  1.] done
[-1.  0. -1. -1. -1.  1. -1.  1. -1.  0.  0. -1. -1.  0.  1.  0.  0.  0.
  1. -1.  0.  0.  1.  1.  1.] done
[-1.  0.  1.  1.  1.  0. -1. -1. -1. -1.  1.  1.  1. -1.  0. -1.  0.  1.
  0. -1. -1. -1.  1. -1.  0.] done
[-1.  1. -1.  1. -1. -1. -1.  0. -1.  0.  0. -1.  0.  0. -1. -1. -1. -1.
  0. -1.  1.  1. -1. -1. -1.] done
[ 1.  1.  1.  1.  0. -1. -1.  0.  0.  1. -1.  0.  1.  0.  1.  1.  1.  1.
  0. -1.  0.  0.  0.  1. -1.] done
[ 0. -1.  0.  1.  0.  1. -1.

In [249]:
#aggregate data of motif counts to find most common motifs across data set
agg_countdata = np.sum(countdata, axis = 0)/6  #divide by 6 because the algorithm will count the same motif spread across the same 3 neurons in every permutation (6 possible)
#for i in range(len(agg_countdata)):
    #print(agg_countdata[i])

#Save aggregate motif count data to a csv file
with open('32random_agg_countdata.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows([str(int(x)) for x in agg_countdata])

In [ ]:
#collect each class's exemplary member
rep_classmembers = partition[:,0]

In [241]:
#back-calculate the representative motif from the class based on index
def visualizeclass(index,rep_classmembers):
    print(rep_classmembers[index])
    
visualizeclass([ 466,  821, 1103, 2212],rep_classmembers)    

[[[-1. -1.  0.]
  [-1.  0.  1.]
  [-1.  0. -1.]]

 [[-1. -1.  0.]
  [ 1. -1.  0.]
  [ 0.  1.  0.]]

 [[-1. -1.  1.]
  [-1.  1. -1.]
  [ 0.  1. -1.]]

 [[-1.  1.  1.]
  [-1.  0. -1.]
  [ 1.  1.  0.]]]


In [258]:
#Find the successful OR motifs' indecies
ORmotifs = [[[-1., -1.,  1.],[ 1., -1.,  0.],[ 0.,  1., -1.]],
[[-1., -1.,  1.],[1., 0., 0.],[ 0.,  0., -1.]],
[[-1., -1.,  1.],[1., 0. ,0.],[ 0.,  1., -1.]],
[[-1., -1.,  1.],[1., 1., 0.],[ 0., -1., -1.]],
[[-1., -1.,  1.],[1., 1., 0.],[ 0.,  0., -1.]],
[[-1.,  0.,  1.],[ 1., -1.,  0.],[ 0.,  0., -1.]],
[[-1.,  0.,  1.],[ 1., -1.,  0.],[ 0.,  1., -1.]],
[[-1.,  0.,  1.],[1., 0., 0.],[ 0., -1., -1.]],
[[-1.,  0.,  1.],[1., 0., 0.],[ 0.,  0., -1.]],
[[-1.,  0.,  1.],[1., 1., 0.],[ 0., -1., -1.]],
[[-1.,  1.,  0.],[-1., -1.,  1.],[ 1.,  0., -1.]],
[[-1.,  1.,  0.],[-1.,  0.,  1.],[ 1., -1., -1.]],
[[-1.,  1.,  0.],[ 0., -1.,  1.],[ 0.,  0., -1.]],
[[-1.,  1.,  0.],[ 0., -1.,  1.],[ 1.,  0., -1.]],
[[-1.,  1.,  0.],[0., 0., 1.],[ 0., -1., -1.]],
[[-1.,  1.,  0.],[0., 0., 1.],[ 1., -1., -1.]],
[[-1.,  1.,  0.],[ 1., -1.,  1.],[-1.,  0., -1.]],
[[-1.,  1.,  0.],[ 1., -1.,  1.],[ 0.,  0., -1.]],
[[-1.,  1.,  0.],[1., 0., 1.],[-1., -1., -1.]],
[[-1.,  1.,  0.],[1., 0., 1.],[ 0., -1., -1.]],
[[-1.,  1.,  1.],[ 1., -1.,  0.],[ 0., -1., -1.]],
[[-1.,  1.,  1.],[ 1., -1.,  0.],[ 0.,  0., -1.]],
[[-1.,  1.,  1.],[1., 0., 0.],[ 0., -1., -1.]],
[[ 0., -1.,  1.],[ 1., -1.,  0.],[-1.,  1., -1.]],
[[ 0., -1.,  1.],[1. ,0., 0.],[-1.,  0., -1.]],
[[ 0., -1.,  1.],[1. ,0., 0.],[-1.,  1., -1.]],
[[ 0., -1.,  1.],[1. ,1., 0.],[-1., -1., -1.]],
[[ 0., -1.,  1.],[1. ,1., 0.],[-1.,  0., -1.]],
[[0. ,0., 1.],[ 1., -1.,  0.],[-1.,  0., -1.]],
[[0. ,0., 1.],[ 1., -1.,  0.],[-1.,  1., -1.]],
[[0. ,0., 1.],[1., 0., 0.],[-1., -1., -1.]],
[[0. ,0., 1.],[1., 0., 0.],[-1.,  0., -1.]],
[[0. ,0., 1.],[1., 1., 0.],[-1., -1., -1.]],
[[0. ,1., 0.],[-1., -1.,  1.],[ 0.,  0., -1.]],
[[0. ,1., 0.],[-1., -1.,  1.],[ 1.,  0., -1.]],
[[0. ,1., 0.],[-1.,  0.,  1.],[ 0., -1., -1.]],
[[0. ,1., 0.],[-1.,  0.,  1.],[ 1., -1., -1.]],
[[0. ,1., 0.],[ 0., -1.,  1.],[-1.,  0., -1.]],
[[0. ,1., 0.],[ 0., -1.,  1.],[ 0.,  0., -1.]],
[[0. ,1., 0.],[0., 0., 1.],[-1., -1., -1.]],
[[0. ,1., 0.],[0., 0., 1.],[ 0., -1., -1.]],
[[0. ,1., 0.],[ 1., -1.,  1.],[-1.,  0., -1.]],
[[0. ,1., 0.],[1. ,0., 1.],[-1., -1., -1.]],
[[0. ,1., 1.],[ 1. ,-1.,  0.],[-1., -1., -1.]],
[[0. ,1., 1.],[ 1. ,-1.,  0.],[-1.,  0., -1.]],
[[0. ,1., 1.],[1. ,0., 0.],[-1., -1., -1.]],
[[1. ,1., 0.],[-1., -1.,  1.],[-1.,  0., -1.]],
[[1. ,1., 0.],[-1., -1.,  1.],[ 0.,  0., -1.]],
[[1. ,1., 0.],[-1.,  0.,  1.],[-1., -1., -1.]],
[[1. ,1., 0.],[-1.,  0.,  1.],[ 0., -1., -1.]],
[[1. ,1., 0.],[ 0., -1.,  1.],[-1.,  0., -1.]],
[[1. ,1., 0.],[0., 0., 1.],[-1., -1., -1.]]]
ORmotifs = np.array(ORmotifs)


In [301]:
flatORmotifs = np.reshape(ORmotifs,(1,52*9))[0]
ormotifsneg1s = 0
ormotifs1s = 0
ormotifs0s = 0
for i in range(52*9):
    if flatORmotifs[i] == 0.:
        ormotifs0s += 1
    if flatORmotifs[i] == 1.:
        ormotifs1s += 1
    if flatORmotifs[i] ==-1.:
        ormotifsneg1s += 1
print(ormotifsneg1s,ormotifs1s,ormotifs0s)    

164 140 164


In [303]:
print(140/4)
164/4


35.0


41.0

In [282]:
ORmotifindecies = []
for i in range(len(ORmotifs)):
    ORmotifindecies.append(lookupclass(partition,ORmotifs[i]))
    if lookupclass(partition,ORmotifs[i])==1440:
        print(ORmotifs[i])
    
ORmotifindecies.sort()

[[-1. -1.  1.]
 [ 1.  0.  0.]
 [ 0.  1. -1.]]
[[-1.  1.  0.]
 [ 0.  0.  1.]
 [ 1. -1. -1.]]
[[ 0.  0.  1.]
 [ 1. -1.  0.]
 [-1.  1. -1.]]
[[ 0.  1.  0.]
 [-1. -1.  1.]
 [ 1.  0. -1.]]


In [284]:
print(ORmotifindecies)

[247, 247, 286, 286, 289, 289, 316, 316, 319, 319, 633, 633, 704, 704, 707, 707, 775, 775, 778, 778, 870, 870, 871, 871, 893, 893, 894, 894, 918, 918, 1373, 1373, 1418, 1418, 1419, 1419, 1437, 1437, 1440, 1440, 1440, 1440, 1504, 1504, 1676, 1676, 1679, 1679, 1715, 1715, 2053, 2053]


In [285]:
ANDlist = [154,163,788,974,1166,1285,1430,2028,2055,2592,3011,3169]
visualizeclass(ANDlist,rep_classmembers)

[[[-1. -1. -1.]
  [-1.  1. -1.]
  [ 1.  0.  0.]]

 [[-1. -1. -1.]
  [-1.  1.  0.]
  [ 0. -1.  1.]]

 [[-1. -1.  0.]
  [ 1. -1. -1.]
  [-1.  1. -1.]]

 [[-1. -1.  0.]
  [ 1.  1.  0.]
  [ 1.  0.  1.]]

 [[-1. -1.  1.]
  [ 0. -1.  0.]
  [-1.  0.  1.]]

 [[-1. -1.  1.]
  [ 0.  1. -1.]
  [-1.  0.  1.]]

 [[-1. -1.  1.]
  [ 1.  0.  0.]
  [-1. -1.  1.]]

 [[-1.  0.  1.]
  [ 0.  1.  0.]
  [ 1.  1.  0.]]

 [[-1.  0.  1.]
  [ 1. -1.  0.]
  [ 0.  1.  1.]]

 [[ 0. -1.  0.]
  [-1.  1.  0.]
  [-1.  1.  0.]]

 [[ 0.  0.  0.]
  [-1.  1. -1.]
  [ 0. -1.  1.]]

 [[ 0.  0.  1.]
  [ 0.  1.  1.]
  [ 1. -1.  1.]]]


In [287]:
#Do any of the OR motifs show up in the larger OR circuits? The larger AND circuits?

or_dist = np.genfromtxt('or_agg_countdata.csv', delimiter=',')
and_dist = np.genfromtxt('and_agg_countdata.csv', delimiter=',')
foo = 0
for i in range(len(ORmotifindecies)):
    if and_dist[ORmotifindecies[i]] != 0:
        foo+=and_dist[ORmotifindecies[i]]
        print(ORmotifindecies[i])
print(foo)

316
316
778
778
894
894
1676
1676
1715
1715
10.0


In [290]:
#How many of the motifs present in the large OR distribution are also present in the large AND distribution?
foo = 0
bar = 0
for i in range(3411):
    if and_dist[i] != 0:
        bar += 1
        if or_dist[i] != 0:
            foo += 1
print(foo) 
print(bar)

52
308


In [269]:

len(ORmotifindecies)

52